In [ ]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu


# 1. Récupère le dataset
df_purchases = dataiku.Dataset("df_purchases")
df = df_purchases.get_dataframe()

# 2. Convertis la colonne de date en datetime
df["transaction_datetime"] = pd.to_datetime(df["transaction_datetime"])
df["month_periode"] = df["transaction_datetime"].dt.to_period('M')
# 3. Définis la date de référence comme la dernière date disponible dans le dataset
reference_date = df["transaction_datetime"].max()  # Date de référence pour le Recency

# 4. Crée un DataFrame pour stocker les métriques RFM par période
rfm_monthly = pd.DataFrame()

# 5. Calcule le Recency pour chaque client (une seule fois)
recency_df = df.groupby("client_id").agg(
    recency=("transaction_datetime", lambda x: (reference_date - x.max()).days)
).reset_index()

# 6. Pour chaque période, calcule Frequency et Monetary
for periode in df["month_periode"].unique():
    periode_end = periode.end_time
    periode_data = df[df["transaction_datetime"] <= periode_end]

    # Frequency et Monetary pour cette période
    f_m_periode = periode_data.groupby("client_id").agg(
        frequency=("transaction_id", "count"),
        monetary=("purchase_sum", "sum")
    ).reset_index()

    # Ajoute la période
    f_m_periode["periode"] = periode

    # Fusionne avec le Recency (calculé une seule fois)
    f_m_periode = f_m_periode.merge(recency_df, on="client_id", how="left")

    # Ajoute au DataFrame global
    rfm_monthly = pd.concat([rfm_monthly, f_m_periode])

# 7. Calcule les scores RFM
rfm_monthly['r_score'] = pd.qcut(rfm_monthly['recency'], 5, labels=[5, 4, 3, 2, 1], duplicates='drop')
rfm_monthly['f_score'] = pd.qcut(rfm_monthly['frequency'], 5, labels=[1, 2, 3, 4, 5], duplicates='drop')
rfm_monthly['m_score'] = pd.qcut(rfm_monthly['monetary'], 5, labels=[1, 2, 3, 4, 5], duplicates='drop')

# 8. Combine les scores RFM
rfm_monthly['rfm_score'] = rfm_monthly['r_score'].astype(str) + rfm_monthly['f_score'].astype(str) + rfm_monthly['m_score'].astype(str)

# 9. Fonction pour assigner les segments
def assign_segment(row):
    if row['r_score'] >= 4 and row['f_score'] >= 4 and row['m_score'] >= 4:
        return "VIP"
    elif row['r_score'] >= 3 and row['f_score'] >= 4 and row['m_score'] >= 3:
        return "Fidèle"
    elif row['r_score'] == 5 and row['f_score'] <= 2 and row['m_score'] >= 3:
        return "Nouveau prometteur"
    elif row['r_score'] <= 2 and row['f_score'] >= 3 and row['m_score'] >= 3:
        return "À réactiver"
    elif row['r_score'] <= 2 and row['f_score'] <= 2 and row['m_score'] <= 2:
        return "Perdu"
    else:
        return "Occasionnel"

# 10. Assigne les segments
rfm_monthly['segment'] = rfm_monthly.apply(assign_segment, axis=1)

# 11. Sauvegarde le dataset RFM
df_RFM_histo = dataiku.Dataset("df_RFM_histo")
df_RFM_histo.write_with_schema(rfm_monthly)


"""
# Read recipe inputs
df_purchases = dataiku.Dataset("df_purchases")
df = df_purchases.get_dataframe()

df["transaction_datetime"] = pd.to_datetime(df["transaction_datetime"])
df["month_periode"] = df["transaction_datetime"].dt.to_period('M')
# Pour chaque client et chaque période, calcule :
# - Recency : jours depuis la dernière transaction AVANT la fin de la période
# - Frequency : nombre de transactions dans la période
# - Monetary : montant total dépensé dans la période

# Crée un DataFrame pour stocker les métriques RFM par période
rfm_monthly = pd.DataFrame()
# pour chaque période, calcule les métriques RFM
for periode in df["month_periode"].unique():
    periode_end = periode.end_time
    periode_data = df[df["transaction_datetime"]<=periode_end]
    rfm_periode = periode_data.groupby("client_id").agg(
        {
            "transaction_datetime":lambda x: (periode_end -x.max()).days,
            "transaction_id":"count",
            "purchase_sum":"sum"
            
            
        }
    ).rename(columns={'transaction_datetime':'recency','transaction_id':'frequency','purchase_sum':'monetary'}).reset_index()
    
    # ajout de la periode
    rfm_periode["periode"] = periode
    rfm_monthly = pd.concat([rfm_monthly,rfm_periode])
# Calcule les scores RFM pour chaque période
rfm_monthly['r_score'] = pd.qcut(rfm_monthly['recency'], 5, labels=[5, 4, 3, 2, 1], duplicates='drop')
rfm_monthly['f_score'] = pd.qcut(rfm_monthly['frequency'], 5, labels=[1, 2, 3, 4, 5], duplicates='drop')
rfm_monthly['m_score'] = pd.qcut(rfm_monthly['monetary'], 5, labels=[1, 2, 3, 4, 5], duplicates='drop')

# Combine les scores
rfm_monthly['rfm_score'] = rfm_monthly['r_score'].astype(str) + rfm_monthly['f_score'].astype(str) + rfm_monthly['m_score'].astype(str)

# Ajoute des segments clairs
def assign_segment(row):
    if row['r_score'] >= 4 and row['f_score'] >= 4 and row['m_score'] >= 4:
        return "VIP"
    elif row['r_score'] >= 3 and row['f_score'] >= 4 and row['m_score'] >= 3:
        return "Fidèle"
    elif row['r_score'] == 5 and row['f_score'] <= 2 and row['m_score'] >= 3:
        return "Nouveau prometteur"
    elif row['r_score'] <= 2 and row['f_score'] >= 3 and row['m_score'] >= 3:
        return "À réactiver"
    elif row['r_score'] <= 2 and row['f_score'] <= 2 and row['m_score'] <= 2:
        return "Perdu"
    else:
        return "Occasionnel"

rfm_monthly['segment'] = rfm_monthly.apply(assign_segment, axis=1)

# Write recipe outputs
df_RFM_histo = dataiku.Dataset("df_RFM_histo")
df_RFM_histo.write_with_schema(rfm_monthly)
"""